# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,900.755608,0.539275,1234,901.294883,900.216333,1802.589765,1800.432666
6,572.071333,-0.298827,1234,571.772506,572.370160,1143.545012,1144.740321
7,3.051897,0.173035,1234,3.224932,2.878861,6.449864,5.757723
8,352.392848,-0.184306,1234,352.208542,352.577154,704.417084,705.154308
9,378.143639,-0.570928,1234,377.572710,378.714567,755.145421,757.429135
10,129.059214,-0.443711,1234,128.615503,129.502925,257.231007,259.005850
11,835.040937,0.539702,1234,835.580639,834.501235,1671.161279,1669.002471
12,920.088088,0.028327,1234,920.116415,920.059761,1840.232831,1840.119523
13,608.203161,0.019377,1234,608.222538,608.183784,1216.445076,1216.367569
14,532.607645,0.754126,1234,533.361771,531.853519,1066.723543,1063.707038


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-901.294883,900.216333
6,-571.772506,572.370160
7,-3.224932,2.878861
8,-352.208542,352.577154
9,-377.572710,378.714567
10,-128.615503,129.502925
11,-835.580639,834.501235
12,-920.116415,920.059761
13,-608.222538,608.183784
14,-533.361771,531.853519


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-901.294883,900.216333
6,-571.772506,572.370160
7,-3.224932,2.878861
8,-352.208542,352.577154
9,-377.572710,378.714567
10,-128.615503,129.502925
11,-835.580639,834.501235
12,-920.116415,920.059761
13,-608.222538,608.183784
14,-533.361771,531.853519


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-901.29488275,  900.21633298],
       [-571.77250623,  572.37016042],
       [  -3.2249318 ,    2.87886126],
       [-352.20854196,  352.57715423],
       [-377.57271043,  378.71456735],
       [-128.61550349,  129.50292502],
       [-835.58063936,  834.50123535],
       [-920.11641546,  920.05976127],
       [-608.2225381 ,  608.18378441],
       [-533.36177134,  531.85351901]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1800.589765,-1800.432666
6,1141.545012,-1144.740321
7,4.449864,-5.757723
8,702.417084,-705.154308
9,753.145421,-757.429135
10,255.231007,-259.005850
11,1669.161279,-1669.002471
12,1838.232831,-1840.119523
13,1214.445076,-1216.367569
14,1064.723543,-1063.707038


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.900756,0.769637
6,0.572071,0.350586
7,0.003052,0.586518
8,0.352393,0.407847
9,0.378144,0.214536
10,0.129059,0.278145
11,0.835041,0.769851
12,0.920088,0.514164
13,0.608203,0.509688
14,0.532608,0.877063


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,900.755608,0.539275
6,572.071333,-0.298827
7,3.051897,0.173035
8,352.392848,-0.184306
9,378.143639,-0.570928
10,129.059214,-0.443711
11,835.040937,0.539702
12,920.088088,0.028327
13,608.203161,0.019377
14,532.607645,0.754126


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
